# cleaning dataset

In [4]:
from __future__ import print_function
import pandas as pd

In [5]:
df = pd.read_csv('/Users/Rutherford/Desktop/dataset.csv',header=None,dtype=str)
df.columns=['error','t1','t2']
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

18
5  misclassified as major


### fix empty strings that are considered nan (floats)

In [6]:
# fix nan's
df.loc[df['t1'].isnull(),'t1']=''
df.loc[df['t2'].isnull(),'t2']=''
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

20
5  misclassified as major


### underscores

In [7]:
df.replace('_{2,}\s?','',regex=True,inplace=True)
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

93
28  misclassified as major


### fillers

In [8]:
fillers = [
        r"\bright\b\s?",
        r"\byou know\b\s?",
        r"\bi think\b\s?",
        r"\bbut i think\b\s?",
        r"\bso to speak\b\s?",
        r"\blike\b\s?",
        r"\bi mean\b\s?",
        r"\bso\b\s?",
        r"\bactually\b\s?",
        r"\bbasically\b\s?",
        r"\bokay\b\s?",
        r"\byeah\b\s?",
        r"\byeah okay\b\s?",
        r"\byes\b\s?",
        r"\bthe\b\s?",
        r"\band\b\s?"]

In [1634]:
#c1 = ['yeah yeah i know','this here is right','yup','righteous','right']
#c2 = ['yeah i know','this here is alright','yep','right-on',"that's"]
#testdf = pd.DataFrame(data={'c1':c1,'c2':c2})

In [1635]:
#testdf['c1'].values

In [1636]:
#testdf['c1'].replace(fillers,'',regex=True).values

In [9]:
df.replace(fillers,'',regex=True,inplace=True)
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

3883
324  misclassified as major


### metas

In [10]:
metas = [
    r"\bchuckle\b\s?",
    r"\blaughter\b\s?",
    r"\bpause\b\s?",
    r"\bnoise\b\s?",
    r"\bmusic\b\s?",
    r"\bapplause\b\s?",
    r"\bvocalization\b\s?",
    r"\bvideo playback\b\s?",
    r"\bautomated voice\b\s?",
    r"\bforeign language\b\s?",
    r"\boverlapping conversation\b\s?",
    r"\bbackground conversation\b\s?",
    r"\bstart-paren\b\s?",
    r"\bend-paren\b\s?"]

In [11]:
df.replace(metas,'',regex=True,inplace=True)
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

4139
334  misclassified as major


### timestamp

In [12]:
df.replace(r"\b\d{6}\b\s?",'',regex=True,inplace=True)
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

9319
340  misclassified as major


### speakers

In [13]:
df.replace(r"\bs\d{0,2}\b\s?",'',regex=True,inplace=True)
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

12195
579  misclassified as major


### trailing whitespaces

In [14]:
df.replace(r" $",'',regex=True,inplace=True)
print(sum(df['t1']==df['t2']))
print(sum(df[df['t1']==df['t2']]['error']=='2'),' misclassified as major')

19216
1221  misclassified as major


### reclassifying

In [15]:
misclassified_indices = df.loc[(df['t1']==df['t2']) & (df['error']=='2')].index.tolist()
# df.loc[misclassified_indices]

In [16]:
# reclassify bad 2's as 1's
print(sum(df['error']=='1'),' total minor errors before')
df.loc[misclassified_indices,'error']='1'
print(sum(df['error']=='1'),' total minor errors after')

56128  total minor errors before
57349  total minor errors after


### manually extracted indices

file saved on desktop as indices.txt

In [1760]:
sum(df.loc[indices,'error']=='2')

436

In [1761]:
# replace manually checked observations with 1's
df.loc[indices,'error']='1'
print(sum(df['error']=='1'),' total minor errors after')

65092  total minor errors after


In [17]:
new_error_col = df['error']

In [19]:
df = pd.read_csv('/Users/Rutherford/Desktop/dataset.csv',header=None,dtype=str)
df.columns=['error','t1','t2']

In [20]:
df['error']=new_error_col

In [21]:
df.to_csv('/Users/Rutherford/Desktop/cleaned_new.csv',header=False,index=False)

### this was to get 'indices'
#### quite a pain

In [696]:
from fuzzywuzzy import fuzz

In [718]:
def close_enough(str1,str2):
    return fuzz.ratio(str1,str2)>=90

In [848]:
potential = df[map(close_enough,df['t1'],df['t2'])]['error']=='2'

In [850]:
potential = potential.index.values

In [852]:
close_calls = df.ix[potential][df['error']=='2']

/Users/Rutherford/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [1266]:
"""test_row_iterator = close_calls.iterrows()
_, test_last = test_row_iterator.next()  # take first item from row_iterator
indices = [227]
def tester_func():
    for i, row in test_row_iterator:
        sys.stdout.flush()
        print(),
        print(row['t1'],'////',row['t2']),

        test_last = row
        choice = input('0 for minor, space for major')
        if choice == 0:
            indices.append(i)
            continue
        elif choice == 1:
            continue
        """